<a href="https://colab.research.google.com/github/komal-shahzada/python/blob/main/fyp__Speech_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install resampy librosa gradio opendatasets
import os
import librosa
import numpy as np
import tqdm
import opendatasets as od
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [11]:
od.download("https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio")

Skipping, found downloaded files in "./ravdess-emotional-speech-audio" (use force=True to force download)


In [12]:
def extract_features(file_path):
    # 2.5 seconds fixed duration  to mach or record live  audio
    data, sample_rate = librosa.load(file_path, res_type='kaiser_fast', duration=2.5, sr=22050, offset=0.5)

    # Padding:  if audio is short add silence to complete it
    if len(data) < 55125:
        data = np.pad(data, (0, 55125 - len(data)), 'constant')

    mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

In [13]:
X, y = [], []
data_path = 'ravdess-emotional-speech-audio'

print("Searching for files and extracting features.. ...")
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith('.wav'):
            try:
                parts = file.split("-")
                emotion = int(parts[2]) # in RAVDESS  3rd part is emotion
                feature = extract_features(os.path.join(root, file))
                X.append(feature)
                y.append(emotion)
            except:
                continue

X = np.array(X)
y = np.array(y)
print(f"Success! {len(X)} files is processed.")

Searching for files and extracting features.. ...
Success! 2880 files is processed.


In [14]:
lb = LabelEncoder()
y_final = to_categorical(lb.fit_transform(y))

X_train, X_test, y_train, y_test = train_test_split(X, y_final, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model ke liye shape set karna
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
print("Data is ready for training!")

Data is ready for training!


In [15]:
model = Sequential([
    Conv1D(256, 8, padding='same', activation='relu', input_shape=(40, 1)),
    BatchNormalization(),
    Conv1D(128, 8, padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling1D(pool_size=8),

    Conv1D(64, 8, padding='same', activation='relu'),
    BatchNormalization(),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_final.shape[1], activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
import gradio as gr
import numpy as np
import librosa

# 1. Emotions Map (same as in training)
emotions_map = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fearful', 7:'disgust', 8:'surprised'}

def predict_emotion_live(audio_path):
    if audio_path is None:
        return "Please record or upload your voice!"

    try:
        # 2. Audio Processing (Settings matching your training)
        data, sr = librosa.load(audio_path, sr=22050, duration=2.5)
        data = librosa.util.normalize(data)
        data, _ = librosa.effects.trim(data, top_db=30)

        # 3. Padding logic
        if len(data) < 55125:
            data = np.pad(data, (0, 55125 - len(data)), 'constant')

        # 4. Feature Extraction
        mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40).T, axis=0)

        # 5. Scaling & Reshaping (Using your trained scaler and model)
        scaled = scaler.transform(mfccs.reshape(1, -1))
        reshaped = np.expand_dims(scaled, axis=2)

        # 6. Prediction
        predictions = model.predict(reshaped, verbose=0)[0]

        # Scores dictionary banana
        results = {}
        for i, prob in enumerate(predictions):
            emotion_label = lb.inverse_transform([i])[0]
            emotion_name = emotions_map[emotion_label]
            results[emotion_name.upper()] = float(prob)

        return results

    except Exception as e:
        return {"Error": str(e)}

# 7. Dashboard Launch (Updated settings for faster loading)
iface = gr.Interface(
    fn=predict_emotion_live,
    inputs=gr.Audio(sources=["microphone", "upload"], type="filepath", label="Record or Upload Audio"),
    outputs=gr.Label(num_top_classes=3, label="Top Predictions"),
    title="🎙️ Komal's Speech Emotion Recognition System",
    description="Speak clearly to see real-time emotion analysis.",
    theme="default"
)

# inline=False for that it never stuck or open in new tab
iface.launch(share=True, inline=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3dfb05b2616b3de493.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
from google.colab import output
from google.colab.output import eval_js
from base64 import b64decode

# Ye JavaScript code get direct acess from browser
def record_audio_colab():
    js = """
    async function recordAudio() {
      const div = document.createElement('div');
      const button = document.createElement('button');
      button.textContent = 'Click to Record (5 Seconds)';
      div.appendChild(button);
      document.body.appendChild(div);
      const stream = await navigator.mediaDevices.getUserMedia({audio:true});
      const recorder = new MediaRecorder(stream);
      const chunks = [];
      recorder.ondataavailable = e => chunks.push(e.data);
      recorder.start();
      await new Promise(resolve => button.onclick = resolve);
      button.textContent = 'Recording... Wait';
      await new Promise(r => setTimeout(r, 5000));
      recorder.stop();
      await new Promise(r => recorder.onstop = r);
      const blob = new Blob(chunks);
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      await new Promise(r => reader.onloadend = r);
      return reader.result;
    }
    """
    display(eval_js(js))
    audio_data = eval_js('recordAudio()')
    return b64decode(audio_data.split(',')[1])

# let start Recording
print("Click on Recording button ...")
audio_bytes = record_audio_colab()
with open("test_mic.wav", "wb") as f:
    f.write(audio_bytes)

# view Prediction result
result = predict_emotion_live("test_mic.wav")
print("\n--- Model Prediction ---")
for emotion, prob in result.items():
    print(f"{emotion}: {prob*100:.1f}%")

Click on Recording button ...


None

/tmp/ipython-input-2615223474.py:14: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sr = librosa.load(audio_path, sr=22050, duration=2.5)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



--- Model Prediction ---
NEUTRAL: 12.1%
CALM: 13.3%
HAPPY: 12.0%
SAD: 12.6%
ANGRY: 13.0%
FEARFUL: 12.0%
DISGUST: 12.1%
SURPRISED: 12.9%
